In [1]:
from loadFilteredData import *
#loading all data
origImg = loadAllTopicData('original')
gingham = loadAllTopicData('gingham')
clarendon = loadAllTopicData('clarendon')
juno = loadAllTopicData('juno')
lark = loadAllTopicData('lark')
gotham = loadAllTopicData('gotham')
reyes = loadAllTopicData('reyes')

['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphitheater
Loading Topic: amusement_park
Loading Topic: aquarium
Loading Topic: aqueduct
Loading Topic: art_gallery
Loading Topic: assembly_line
Loading Topic: auditorium
(9000, 128, 128, 3)
['abbey', 'airport_terminal', 'amphitheater', 'amusement_park', 'aquarium', 'aqueduct', 'art_gallery', 'assembly_line', 'auditorium']
Loading Topic: abbey
Loading Topic: airport_terminal
Loading Topic: amphithe

In [4]:
def createDataPlaces(images, trainPercentage, validationPercentage):
    categories = len(images)

    imgList = []
    vectors = []

    testImgList = []
    testVectors = []

    valImgList = []
    valVectors = []
    

    #data for original image
    for c in range(categories):
        numImages = images[c].shape[0]
        print(numImages)
        numTrain = int(numImages * trainPercentage)
        numVal = int(numTrain * validationPercentage)
        
        imgList.append(images[c][:numTrain - numVal])
        
        valImgList.append(images[c][numTrain - numVal:numTrain])

        featureVector = np.zeros((numTrain - numVal, categories))
        featureVector[:, c] = 1
        vectors.append(featureVector)
        
        featureVector = np.zeros((numVal, categories))
        featureVector[:, c] = 1
        valVectors.append(featureVector)
    
        #testing data
        testImgList.append(images[c][numTrain:])

        featureVector = np.zeros((numImages - numTrain, categories))
        featureVector[:, c] = 1
        testVectors.append(featureVector)


    X = np.vstack(imgList)
    y = np.vstack(vectors)

    Xtest = np.vstack(testImgList)
    ytest = np.vstack(testVectors)
    
    Xval = np.vstack(valImgList)
    yval = np.vstack(valVectors)

    X, y = shuffle(X, y, random_state=0)
    Xtest, ytest = shuffle(Xtest, ytest, random_state=0)
    Xval, yval = shuffle(Xval, yval, random_state=0)
    
    return X, y, Xtest, ytest, Xval, yval
X, y, Xtest, ytest, Xval, yval = createDataPlaces([origImg, clarendon, gingham, juno, lark, gotham, reyes], .9, .005)
# X, y, Xtest, ytest, Xval, yval = createDataPlaces([origImg, juno], .9, .005)

9000
9000
9000
9000
9000
9000
9000


In [5]:
print(X.shape, y.shape)
print(Xtest.shape, ytest.shape)
print(Xval.shape, yval.shape)

(56420, 128, 128, 3) (56420, 7)
(6300, 128, 128, 3) (6300, 7)
(280, 128, 128, 3) (280, 7)


In [6]:
def splitImg(X, y):
    Xsmall = []
    ysmall = []
    for i in range(X.shape[0]):
        l = 0
        for j in range(0, X.shape[1], X.shape[1] // 4):
            for k in range(0, X.shape[1], X.shape[1] // 4):
                block = X[i, j : j + 32, k : k + 32]
                Xsmall.append(block)
                ysmall.append(y[i])
                l += 1
    Xsmall = np.array(Xsmall)
    ysmall = np.array(ysmall)
    return Xsmall, ysmall

In [9]:
Xsmall, ysmall = splitImg(X, y)
Xval_small, yval_small = splitImg(Xval, yval)

In [11]:
print(Xsmall.shape, ysmall.shape)
print(Xval_small.shape, yval_small.shape)

(902720, 32, 32, 3) (902720, 7)
(4480, 32, 32, 3) (4480, 7)


In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import LeakyReLU
import numpy as np

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(32, 32, 3), strides=2))
model.add(LeakyReLU(alpha=0.3))
model.add(Conv2D(32, (3, 3), strides = 2))
model.add(LeakyReLU(alpha=0.3))
model.add(MaxPooling2D(pool_size=3, strides =2))
model.add(Flatten())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.3))
model.add(Dense(256))
model.add(LeakyReLU(alpha=0.3))
# model.add(Dropout(0.5))
model.add(Dense(7, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Using TensorFlow backend.


In [56]:
model.fit(Xsmall, ysmall, epochs=5, batch_size=4096, validation_data=(Xval_small, yval_small))

Train on 902720 samples, validate on 4480 samples
Epoch 1/5
902720/902720 [==============================] - 46s 50us/step - loss: 0.3736 - acc: 0.8555 - val_loss: 0.4064 - val_acc: 0.8391
Epoch 2/5
902720/902720 [==============================] - 45s 50us/step - loss: 0.3853 - acc: 0.8518 - val_loss: 0.3745 - val_acc: 0.8500
Epoch 3/5
902720/902720 [==============================] - 46s 51us/step - loss: 0.3693 - acc: 0.8573 - val_loss: 0.3827 - val_acc: 0.8484
Epoch 4/5
902720/902720 [==============================] - 47s 52us/step - loss: 0.3759 - acc: 0.8545 - val_loss: 0.3990 - val_acc: 0.8424
Epoch 5/5
902720/902720 [==============================] - 46s 51us/step - loss: 0.3658 - acc: 0.8582 - val_loss: 0.3927 - val_acc: 0.8433


In [14]:
Xtest_small, ytest_small = splitImg(Xtest, ytest)

In [57]:
predict = model.predict(Xtest_small)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest_small.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest_small.argmax(axis=1), predict.argmax(axis=1)))
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(ytest_small.argmax(axis=1), predict.argmax(axis=1))

[[12206   592   295   187   280   360   480]
 [ 1565 11127    76   123   180   888   441]
 [   31     7 14158     7     9    44   144]
 [  141    46   105 11470  2252   132   254]
 [  279    77   112  3777  9730    74   351]
 [  230   383    98   158    52 13401    78]
 [  426   101   499    70   142    56 13106]]
0.845218253968254


(array([0.82040597, 0.90221357, 0.92276608, 0.72631712, 0.7694741 ,
        0.89608826, 0.88232126]),
 array([0.84763889, 0.77270833, 0.98319444, 0.79652778, 0.67569444,
        0.930625  , 0.91013889]),
 array([0.83380012, 0.83245427, 0.95202232, 0.75980392, 0.7195415 ,
        0.91303015, 0.89601422]),
 array([14400, 14400, 14400, 14400, 14400, 14400, 14400], dtype=int64))

In [58]:
from scipy.stats import mode
predictOrig = []
print(predict.shape)
for i in range(0, predict.shape[0], 16):
    votes = np.argmax(predict[i: i+16], axis=1)
    vote = mode(votes)
    predictOrig.append(vote[0])
predictOrig = np.array(predictOrig)

(100800, 7)


In [59]:
print(ytest.shape)
print(predictOrig.shape)

(6300, 7)
(6300, 1)


In [60]:
from sklearn.preprocessing import OneHotEncoder
onehot_encoder = OneHotEncoder(sparse=False)
predictVote = onehot_encoder.fit_transform(predictOrig)
print(predictVote.shape)

(6300, 7)


C:\Users\Chunlok Lo\AppData\Local\conda\conda\envs\magiclearning\lib\site-packages\sklearn\preprocessing\_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [61]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest.argmax(axis=1), predictVote.argmax(axis=1)))
from sklearn.metrics import accuracy_score
print(accuracy_score(ytest.argmax(axis=1), predictVote.argmax(axis=1)))
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(ytest.argmax(axis=1), predictVote.argmax(axis=1))

[[877   3   8   1   5   1   5]
 [ 24 858   1   0   1  11   5]
 [  1   0 899   0   0   0   0]
 [  0   0   3 867  29   0   1]
 [  2   0   4 181 711   1   1]
 [  0   0   0   2   0 898   0]
 [  0   0   8   0   0   0 892]]
0.9526984126984127


(array([0.97013274, 0.99651568, 0.97399783, 0.82492864, 0.95308311,
        0.98572997, 0.98672566]),
 array([0.97444444, 0.95333333, 0.99888889, 0.96333333, 0.79      ,
        0.99777778, 0.99111111]),
 array([0.97228381, 0.97444634, 0.98628634, 0.88877499, 0.86391252,
        0.99171728, 0.98891353]),
 array([900, 900, 900, 900, 900, 900, 900], dtype=int64))

In [62]:
from keras.models import load_model
model.save('FullNNFilter4.h5')